---
## Fehlende Werte mit PANDAS

In [1]:
import pandas as pd
from io import StringIO

In [2]:
csv_data = \
    '''A,B,C,D
        1.0,2.0,3.0,4.0
        5.0,6.0,,8.0
        10.0,11.0,12.0,
    '''
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


### Fehlende Werte finden
`1. isna()`       : Liefert `True` zurück, in den Zellen, die n/a haben<br>
`2. isna().sum()` : Gibt je Spalte die Summe als `int` zurück

In [3]:
df.isna()

,A,B,C,D
0,False,False,False,False
1,False,False,True,False
2,False,False,False,True


In [4]:
df.isna().sum()

A    0
B    0
C    1
D    1
dtype: int64

### Die meisten SKLEARN Bibliotheken können mit Pandas arbeiten, sind aber auf NUMPY spezialisiert 
Sprich man sollte möglichst Numpy Arrays übergeben, die man via `df.values` erhalten kann

In [5]:
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

In [6]:
type(df.values)

numpy.ndarray

---
### 1. fehlende Features (Merkmale) oder Zeilen (Instanzen) komplett entfernen

In [7]:
# Spalte komplett entfernen
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [8]:
# Zeile komplett entfernen
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [9]:
# Nur Zeilen löschen, in denen alle Spalten N/A enthalten -> gibt ganzes Array zurück, weil keine Zeile durchgängig nur N/A enthält
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [10]:
# Treshhold verlangt eine Mindestanzahl von reelen Zahlen in der Zeile --> löscht beide Zeilen, in denen das nicht der Fall ist
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [11]:
# Subset löscht nur Zeilen, in denen N/A in einer definierten Spalte enthalten ist
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


---
## IMPUTATION TECHNICS

### 1. Imputation mit gesamt-DF MEAN() Wert

In [13]:
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


### 2. Eine Spalte die N/A mit dem MEAN() derselben Spalte auffüllen

In [16]:
df['C'] = df['C'].fillna(df['C'].mean())
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,NaN


### 3. Handhabung kategorialer Variablen 
Man muss zwischen `a) ordinalen` und `b) nominalen` Merkmalen unterscheiden:<br>
<br>
`ad a)` natürliche Reihenfolge, können sortiert werden - aber Abstände zwischen den einzelnen Werten sind nicht quantifizierbar (es kann nicht mit ihnen “normal gerechnet” werden. Beispiel: Schulnoten, Präferenzrangfolgen, etc.<br>
<br>
`ad b)` Daten, die in keinerlei natürliche Reihenfolge gebracht werden können – beispielsweise um das Geschlecht, die Haarfarbe oder die Telefonnummer